In [118]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


ModuleNotFoundError: No module named 'regex'

In [2]:
# grab local environmental files (client id, client secret)
import os
from dotenv import load_dotenv
load_dotenv()

True

Finding Billboard Top 50 (not 100, so Spotify API doesn't hate us)


In [52]:
url = 'https://www.billboard.com/charts/hot-100/2022-05-21/'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
raw_song_names = soup.find_all('h3', attrs = {'id':'title-of-a-story','class':'a-no-trucate'})
raw_artists = soup.find_all('span', attrs = {'class':'c-label','class':'a-no-trucate'})


In [119]:
song_title = []
song_artist = []

for raw_name in raw_song_names:
  song_title.append(str(raw_name.get_text().strip()))

for raw_artist in raw_artists:
  song_artist.append(str(raw_artist.get_text().replace("Featuring"," ").strip()))

In [120]:
billboard_db = pd.DataFrame()
billboard_db['Titles'] = song_title
billboard_db['Artists'] = song_artist
billboard_db.drop(billboard_db.tail(50).index,inplace=True)

In [121]:
billboard_db

,Titles,Artists
0,First Class,Jack Harlow
1,As It Was,Harry Styles
2,Wait For U,Future Drake & Tems
3,Moscow Mule,Bad Bunny
4,Titi Me Pregunto,Bad Bunny
5,Despues de La Playa,Bad Bunny
6,Heat Waves,Glass Animals
7,Big Energy,Latto
8,About Damn Time,Lizzo
9,Me Porto Bonito,Bad Bunny & Chencho Corleone


In [7]:
#Auth keys are stored locally in a .env file, to keep hidden.
CL_id = os.getenv('CL_ID')
CL_secret = os.getenv('CL_SECRET')
AUTH_URL = 'https://accounts.spotify.com/api/token'


In [56]:
#client credentials flow

#request authorization
auth_resp = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CL_id,
    'client_secret': CL_secret,
})

# convert the response to JSON
auth_resp_data = auth_resp.json()

# save the access token
access_token = auth_resp_data['access_token']

In [117]:
access_token

'BQB2ZN1GNXbl71P1JJjOj_P-RcNTuwvdRJvtNIDU7N4byqciC5N1wG_mR1KnwnAoPpHEAY_PRoU2a-WYE94'

In [57]:
# using the access token

#endpoint to hit 
endpoint = 'https://api.spotify.com/v1/search?q='

#creating the headers
headers = {
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json'
}

In [59]:
#helper function for get_song_info to convert duration from miliseconds to minute/second readable format
def song_time(duration):
  total_seconds = int(duration / 1000)
  minutes       = int(total_seconds / 60)
  seconds       = int(total_seconds - minutes * 60)
  return ("{}:{:02}".format(minutes, seconds))

In [122]:
# Helper function to create a dataframe based on the information that we want to receive:
# the popularity score (out of 100), explicity (safe for school?), the duration in minute/seconds, release date, and a preview if you want to listen
# WARNING TO TURN YOUR AUDIO DOWN IF YOU CLICK ON PREVIEW (default sound setting is LOUD).

def get_song_info(data):
  #values that we're looking for
  popularity = []
  release_date = []
  explicit = []
  duration = []
  preview_url = []

  #iterate through the json returned to extract the information and append to empty lists
  for track_info in data['tracks']['items']:
    popularity.append([track_info][0]['popularity'])
    release_date.append([track_info][0]['album']['release_date'])
    explicit.append([track_info][0]['explicit'])
    preview_url.append([track_info][0]['preview_url'])

    dur = [track_info][0]['duration_ms']
    min_sec_dur = str(song_time(dur))
    duration.append(min_sec_dur)

  #setting up our dataframe
  album = {
  'popularity': popularity,
  'release_date': release_date,
  'explicit': explicit,
  'duration': duration,
  'preview_url': preview_url
  }
  album_df = pd.DataFrame(album)

  return album_df

In [123]:
def get_spot_info(title,artist):
  r = requests.get(endpoint + artist + ' ' + title + '&type=track' + '&limit=1',headers=headers)
  data = r.json()
  return get_song_info(data=data)


In [124]:
spotify_df = pd.DataFrame()

for i in billboard_db.index:
  title = billboard_db['Titles'][i]
  artist = billboard_db['Artists'][i]
  df = get_spot_info(title=title,artist=artist)
  spotify_df = pd.concat([df,spotify_df],ignore_index=True)



In [125]:
spotify_df

,popularity,release_date,explicit,duration,preview_url
0,81.0,2022-05-06,False,4:10,None
1,92.0,2022-05-06,True,4:05,https://p.scdn.co/mp3-preview/585736d2d4dba1ee...
2,64.0,2022-04-22,False,3:16,https://p.scdn.co/mp3-preview/0a71593f5365a725...
3,86.0,2021-06-25,True,3:30,https://p.scdn.co/mp3-preview/c707247354131537...
4,86.0,2021-10-14,False,3:44,https://p.scdn.co/mp3-preview/3ca514309c910bb7...
5,86.0,2022-05-06,False,3:28,https://p.scdn.co/mp3-preview/32b762468315fd6a...
6,85.0,2022-05-06,False,3:30,https://p.scdn.co/mp3-preview/b3b4baf267156c50...
7,84.0,2022-01-28,False,2:49,None
8,89.0,2021-09-17,True,2:23,https://p.scdn.co/mp3-preview/d62f4961618e4b4d...
9,91.0,2021-08-13,True,2:48,https://p.scdn.co/mp3-preview/83c53804d9a84bee...


In [126]:
test_df = pd.DataFrame

title = billboard_db['Titles'][45]
artist = billboard_db['Artists'][45]
df = get_spot_info(title=title,artist=artist)
# test_df = pd.concat([df,test_df])

In [127]:
print(title,artist)

Easy On Me Adele


In [128]:
df

,popularity,release_date,explicit,duration,preview_url
0,86,2021-10-14,False,3:44,https://p.scdn.co/mp3-preview/3ca514309c910bb7...


In [112]:
spotify_df

,popularity,release_date,explicit,duration,preview_url
0,81.0,2022-05-06,False,4:10,None
1,92.0,2022-05-06,True,4:05,https://p.scdn.co/mp3-preview/585736d2d4dba1ee...
2,64.0,2022-04-22,False,3:16,https://p.scdn.co/mp3-preview/0a71593f5365a725...
3,86.0,2021-06-25,True,3:30,https://p.scdn.co/mp3-preview/c707247354131537...
4,86.0,2021-10-14,False,3:44,https://p.scdn.co/mp3-preview/3ca514309c910bb7...
5,86.0,2022-05-06,False,3:28,https://p.scdn.co/mp3-preview/32b762468315fd6a...
6,85.0,2022-05-06,False,3:30,https://p.scdn.co/mp3-preview/b3b4baf267156c50...
7,84.0,2022-01-28,False,2:49,None
8,89.0,2021-09-17,True,2:23,https://p.scdn.co/mp3-preview/d62f4961618e4b4d...
9,91.0,2021-08-13,True,2:48,https://p.scdn.co/mp3-preview/83c53804d9a84bee...


In [48]:
billboard_db = pd.concat([billboard_db,spotify_df])
billboard_db.head()

,popularity,release_date,explicit,duration,preview_url,Titles,Artists
0,89.0,2021-07-09,True,2:21,https://p.scdn.co/mp3-preview/40a8daf1f3cbb5c6...,NaN,NaN
0,70.0,2021-09-09,False,2:59,None,NaN,NaN
0,74.0,2022-05-06,True,3:44,https://p.scdn.co/mp3-preview/b79404b67573c396...,NaN,NaN
0,74.0,2022-04-28,True,2:59,None,NaN,NaN
0,89.0,2021-11-04,False,2:44,https://p.scdn.co/mp3-preview/bf5bc1080a2416f7...,NaN,NaN


In [ ]:
#creating empty dataframe
artist_df = pd.DataFrame()

#iterating through artist_ids list to call get_artist function on all values in the list and creating a new dataframe comprised of everything.
for i in range(len(artist_ids)):
  df = get_artist(artist_ids[i])
  artist_df = pd.concat([artist_df,df])

In [70]:
def get_song_info():
  r = requests.get(endpoint + artist + ' ' + title + '&type=track' + '&limit=1',headers=headers)
  data = r.json()
  return get_song_info(data)

In [16]:
get_song_info(data)

,popularity,release_date,explicit,duration,preview_url
0,94,2022-04-08,True,2:53,https://p.scdn.co/mp3-preview/27913e771fb16c54...


In [ ]:
#popularity
#explicit
#preview_url
#duration
#release_date

In [ ]:
def get_artist(id):
  #sends a GET request to our spotify endpoint with the ID given and our aforementioned header
  r = requests.get(endpoint + 'artists/' + id + '/albums', headers=headers)
  data = r.json()
  return get_album_info(data)

In [ ]:
#search url = "https://api.spotify.com/v1/search?q=Linkin+Park&type=track&limit=5&access_token=\(token)"  
# pass the token in this string thats it..
